In [1]:
import pandas as pd

In [ ]:
df = pd.read_json(filepath, lines=True)
df = pd.read_json('data/log_data/2018/11/2018-11-01-events.json', lines=True)

# log dataset
log_data/2018/11/2018-11-12-events.json
log_data/2018/11/2018-11-13-events.json

# songs dataset
song_data/A/B/C/TRABCEI128F424C983.json
song_data/A/A/B/TRAABJL12903CDCF1A.json

In [15]:
df = pd.read_json("data/song_data/A/B/C/TRABCFL128F149BB0D.json", lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARLTWXK1187FB5A3F8,32.74863,"Fort Worth, TX",-97.32925,King Curtis,326.00771,1,SODREIN12A58A7F2E5,A Whiter Shade Of Pale (Live @ Fillmore West),0


In [12]:
df['artist_id'].nunique()

1

In [1]:
%run -i 'sql_queries.py'

In [2]:
%run -i 'create_tables.py'

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
# df = pd.read_json('/home/workspace/data/log_data/2018/11/2018-11-01-events.json', lines=True)
def process_log_file(cur, filepath):
    # open log file
    df = pd.read_json(filepath, lines=True)

    # filter by NextSong action
    df = df[df['page']=='NextSong']

    # convert timestamp column to datetime
    df['datetime'] = df['ts'].apply(lambda x: pd.to_datetime(x))  
    
    # insert time data records
    df['time_data'] = df['datetime'].apply(lambda x: [x
                                                      ,pd.to_datetime(x).hour
                                                      ,pd.to_datetime(x).day
                                                      ,pd.to_datetime(x).week
                                                      ,pd.to_datetime(x).month
                                                      ,pd.to_datetime(x).year
                                                      ,pd.to_datetime(x).weekday()])
    time_data_df=pd.DataFrame(df['time_data'])
    time_data_df[['start_time', 'hour', 'day', 'week', 'month', 'year','weekday']]=pd.DataFrame(time_data_df.time_data.tolist()
                                                                                                ,index= time_data_df.index)

#     column_labels = 
    time_df = time_df = time_data_df[['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']]

    for i, row in time_df.iterrows():
        cur.execute(time_table_insert, list(row))

    # load user table
    user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

    # insert user records
    for i, row in user_df.iterrows():
        cur.execute(user_table_insert, row)

    # insert songplay records
    for index, row in df.iterrows():
        
        # get songid and artistid from song and artist tables
        cur.execute(song_select, (row.song, row.artist, row.length))
        results = cur.fetchone()
        
        if results:
            songid, artistid = results
        else:
            songid, artistid = None, None

        # insert songplay record
        songplay_data = (index, row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
        cur.execute(songplay_table_insert, songplay_data)

In [ ]:
# insert time data records
    time_data = df['ts'].apply(lambda x: [x
                                          ,pd.to_datetime(x).hour
                                          ,pd.to_datetime(x).day
                                          ,pd.to_datetime(x).week
                                          ,pd.to_datetime(x).month
                                          ,pd.to_datetime(x).year
                                          ,pd.to_datetime(x).weekday()])
    t=pd.DataFrame(df['t'])
    t[['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']]=pd.DataFrame(t.t.tolist()
                                                                                      ,index= t.index)
#     column_labels = 
    time_df = t[['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']]

    for i, row in time_df.iterrows():
        cur.execute(time_table_insert, list(row))

    # load user table
    user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

    # insert user records
    for i, row in user_df.iterrows():
        cur.execute(user_table_insert, row)

In [ ]:
    # insert time data records
    time_data = df['ts'].apply(lambda x: [x
                                          ,pd.to_datetime(x).hour
                                          ,pd.to_datetime(x).day
                                          ,pd.to_datetime(x).week
                                          ,pd.to_datetime(x).month
                                          ,pd.to_datetime(x).year
                                          ,pd.to_datetime(x).weekday()])
    t=pd.DataFrame(df['t'])
    t[['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']]=pd.DataFrame(t.t.tolist()
                                                                                      ,index= t.index)
#     column_labels = 
    time_df = t[['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']]

    for i, row in time_df.iterrows():
        cur.execute(time_table_insert, list(row))

    # load user table
    user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

    # insert user records
    for i, row in user_df.iterrows():
        cur.execute(user_table_insert, row)

In [32]:
song_data.values.flatten().tolist()

['SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008, 149.86404]

In [4]:
def process_song_file(cur, filepath):
    # open song file
    df = pd.read_json(filepath, lines=True)

    # insert song record
    song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']]
    cur.execute(song_table_insert, song_data.values.flatten().tolist())
    
    # insert artist record
    artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
    cur.execute(artist_table_insert, artist_data.values.flatten().tolist())

In [5]:
def process_data(cur, conn, filepath, func):
    # get all files matching extension from directory
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))

    # get total number of files found
    num_files = len(all_files)
    print('{} files found in {}'.format(num_files, filepath))

    # iterate over files and process
    for i, datafile in enumerate(all_files, 1):
        func(cur, datafile)
        conn.commit()
        print('{}/{} files processed.'.format(i, num_files))

In [6]:
def main():
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
    cur = conn.cursor()

#     process_data(cur, conn, filepath='data/song_data', func=process_song_file)
    process_data(cur, conn, filepath='data/log_data', func=process_log_file)

    conn.close()

In [ ]:
if __name__ == "__main__":
    main()